In [ ]:
%matplotlib widget

In [ ]:
import os
import fcGMM as gmm
import matplotlib.pyplot as plt
import numpy as np
import fcsparser
import pandas as pd

# Check if the fcs files are in the data folder

List the files in the data folder, you should see the fcs files you want to analyze.

In [ ]:
%ls data

# Check the config file

In [ ]:
%ls *.dat

In [ ]:
!more dataCTV.dat

In [ ]:
dataDir = 'data/'
fcsFileName = '29112021 high FSC_T 1 Day 1 pm.fcs'

# channel names
xlabel = 'V450-A'
ylabel = None
zlabel = None

# set dimensions for gmm
dim = 1

# Run gatting strategy

The gate parameters should be adjusted for your specific gatting strategy

In [ ]:
# gate paramenters
th = 10**-11 #kernel density th
fsca_lb = 47000
fsca_hb = 250000
ssca_lb = 5000
ssca_hb = 250000

In [ ]:
meta, data = fcsparser.parse(dataDir+fcsFileName, meta_data_only=False, reformat_meta=True)
print('data keys ',data.keys)

x,y,z,data = gmm.removeZeros(data,xlabel,ylabel,zlabel)
ssca = data['SSC-A']
fsca = data['FSC-A']

hss,hfs = ssca_hb, fsca_hb
lss,lfs = ssca_lb , fsca_lb
gate1 = (fsca<lfs)|(fsca>hfs)
gate2 = (ssca<lss)|(ssca>hss)

xx, yy,f,kernel = gmm.getKernel(fsca,ssca)

th = th
v = -np.log10(th)

fig, ax = plt.subplots(1,1)
#plt.subplots_adjust(left=0.15, bottom=0.35)
xy = np.vstack([fsca,ssca])
valK = kernel(xy)
ax.scatter(fsca,ssca,s=1)
ax.set_xlabel('fsca')
ax.set_ylabel('ssca')
ax.scatter(fsca,ssca,c=valK,s=1)
ax.scatter(fsca[valK>th],ssca[valK>th],c=valK[valK>th])
ax.scatter(fsca[gate1],ssca[gate1],c='red',s=1)
ax.scatter(fsca[gate2],ssca[gate2],c='yellow',s=1)
ax.axvline(fsca_hb)
ax.axvline(fsca_lb)
ax.axhline(ssca_hb)
ax.axhline(ssca_lb)
ax.set_title(fcsFileName)

gateTh = (valK<th)
graph,dfTh = gmm.prepData(x,y,z,ssca,fsca,gateTh,gate1,gate2,None,xlabel,ylabel,zlabel)

In [ ]:
dfTh

In [ ]:
x

# Run Gaussian Mixture

In [ ]:
lx = 'log '+xlabel # V450-A'
ly = None
lz = None

time = '24h '

df,x = gmm.getCols(dfTh,lx,ly,lz)
gmm.plot1dlog(x, time, llim=1, lim = 18, xlabel=lx, nbins = 400.0)

Set the m-Gaussians means and variances (Cs) 

In [ ]:
m = 3
hour = '24'
sufx = ''

res0 = pd.DataFrame( {'means 0':[15.1],'Cs V450-A 0':[0.13],
                      'means 1':[14.3],'Cs V450-A 1':[0.05],
                      'means 2':[13.4],'Cs V450-A 2':[0.08]})

print(res0)

Run the GMM the first plot shows the gaussians fit, the second the prior probability of each gaussian

In [ ]:
means,Cs,ws = gmm.runGM(hour,dim,m,dfTh,res0,sufx,outf=True,show=False,cond=False,xlabel=xlabel,ylabel=ylabel,zlabel=zlabel)
plt.show()

dfW = dfTh.copy()

for k in range(m):
    dfW.loc[:,'w'+str(k)] = ws[k]

In [ ]:
dfW